In [31]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-04-30 17:01:07--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-04-30 17:01:07 (19.8 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [33]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()
print(len(text))

1115394


In [34]:
print(text[:2])

Fi


In [35]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [36]:
# mapping to int 
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("hii there"))
print(decode(encode("hii there")))
# Small voc -> long sequences 

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [37]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:2]) 

torch.Size([1115394]) torch.int64
tensor([18, 47])


In [38]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [39]:
block_size = 8 # or context length
train_data[:block_size+1] # 8 examples 

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [40]:
x = train_data[:block_size+1]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")
# As little as one and long as block_size # 

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [41]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data 
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y 

xb, yb = get_batch('train')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")

torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1, 58, 46] the t

In [50]:
import torch
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx) # (B, T, C)
    # Dense vectors -> defined size #
    # Weights leanred               #
    # Relationship                  #
    # word2vec                      #
    # voc + size vector             #
    # What comes next               #
    # No context see                #
    # -ln(1/65)                     #
    
    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C) # Strech B*T
      targets = targets.view(B*T) # or -1 
      loss = F.cross_entropy(logits, targets)

    return logits, loss 
  

  def generate(self, idx, max_new_tokens):
    # idx -> B, T # Context in batch #
    for _ in range(max_new_tokens):
      logits, loss = self(idx) # (B*T, C)
      logits = logits[:, -1, :] # Last element in T -> (B, C)
      # History not used #
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(logits[0, 0].max())
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(-1.5101, grad_fn=<MaxBackward1>)
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [51]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [56]:
batch_size = 32
for steps in range(10000):

  xb, yb = get_batch('train')

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.4202239513397217


In [57]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


The fas; pseamiourild, joner t; aiutin idon am, imifive ms y sald IFFon mis f ket me ck sad and alsp


In [58]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.rand(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [61]:
xbow = torch.zeros((B,T,C)) # Bag of words
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1]
    xbow[b, t] = torch.mean(xprev, 0)

In [71]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (((B)), T, T) @ (B, T, C) -> (B, T, C)
xbow2

tensor([[[0.0783, 0.4956],
         [0.3507, 0.4590],
         [0.3006, 0.3156],
         [0.3717, 0.4108],
         [0.3326, 0.3806],
         [0.3953, 0.4140],
         [0.3470, 0.4644],
         [0.4134, 0.4368]],

        [[0.6005, 0.7079],
         [0.5554, 0.5572],
         [0.6657, 0.4908],
         [0.7234, 0.6090],
         [0.5817, 0.6344],
         [0.6161, 0.6865],
         [0.5946, 0.7081],
         [0.5362, 0.6462]],

        [[0.2944, 0.3677],
         [0.3887, 0.5215],
         [0.4333, 0.5322],
         [0.4675, 0.4611],
         [0.4948, 0.5105],
         [0.4255, 0.5525],
         [0.4748, 0.4842],
         [0.4875, 0.5094]],

        [[0.9100, 0.7684],
         [0.8118, 0.4135],
         [0.7959, 0.5978],
         [0.8454, 0.6482],
         [0.6993, 0.6530],
         [0.5973, 0.6449],
         [0.5726, 0.5670],
         [0.5115, 0.5632]]])

In [62]:
x[0]

tensor([[0.0783, 0.4956],
        [0.6231, 0.4224],
        [0.2004, 0.0287],
        [0.5851, 0.6967],
        [0.1761, 0.2595],
        [0.7086, 0.5809],
        [0.0574, 0.7669],
        [0.8778, 0.2434]])

In [63]:
xbow[0]

tensor([[0.0783, 0.4956],
        [0.3507, 0.4590],
        [0.3006, 0.3156],
        [0.3717, 0.4108],
        [0.3326, 0.3806],
        [0.3953, 0.4140],
        [0.3470, 0.4644],
        [0.4134, 0.4368]])

In [72]:
torch.allclose(xbow, xbow2)

True

In [65]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [67]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdims=True) # For average 
b = torch.randint(0, 10, (3,2)).float() # Sum or cols -> repeating without tril
c = a @ b
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [76]:
# Version 3
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T)) # Affinity -> might not be "1"
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1) # exp + normalize 
xbow3 = wei @ x 
torch.allclose(xbow, xbow3)


True

In [95]:
torch.manual_seed(1337)
B,T,C = 4, 8, 32 # batch, time, channels

# Head
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x)
# I am a query -> looking etc -> find similar keys 
wei = q @ k.transpose(-2, -1) # (B, T, T)

x = torch.randn(B,T,C)
tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ v # 16 -> size head. 
# Q -> interest, K -> Have, V -> give  
#out = wei @ x  # 32
out.shape
# Also cyclic to it self element 
# Attention communication mechanism -> graph 
# Attention -> no position 
# No talk with other element in batch 
# Sentiment -> encoder or QnA
# Self-attention -> x -> k, q, v 
# k, v -> can come from sth else e.g. encoder, db 

torch.Size([4, 8, 16])

In [93]:
wei[0]

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.8557e-01, 1.1443e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [6.0002e-02, 2.4910e-01, 6.9090e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [3.6545e-03, 6.6198e-01, 3.1626e-01, 1.8103e-02, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [4.2962e-04, 3.0372e-01, 1.0769e-01, 5.8706e-01, 1.0975e-03, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [3.7807e-02, 2.5455e-02, 1.7927e-01, 5.0882e-01, 1.0452e-01, 1.4413e-01,
         0.0000e+00, 0.0000e+00],
        [1.2364e-01, 1.8130e-01, 1.1670e-02, 5.5879e-02, 5.2206e-01, 5.3096e-02,
         5.2346e-02, 0.0000e+00],
        [7.1010e-01, 1.5177e-01, 1.7532e-02, 9.9237e-04, 1.0815e-02, 4.1076e-02,
         5.2606e-02, 1.5108e-02]], grad_fn=<SelectBackward0>)

In [98]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1)# * head_size**-0.5
# No picky softmax 

In [99]:
k.var()

tensor(1.0966)

In [100]:
q.var()

tensor(0.9416)

In [101]:
wei.var()

tensor(16.1036)

In [102]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [103]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

Layer norm on x now not like in the paper + dropout before layernorm. Cross-attention. Translation -> <start> + encoder  

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features